In [1]:
!pip install pandas numpy scikit-learn joblib requests beautifulsoup4

DEPRECATION: Loading egg at c:\users\hp\anaconda3\lib\site-packages\huggingface_hub-0.29.1-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
import pandas as pd
import numpy as np
import re
import joblib  # For saving/loading the trained model
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [4]:
# Load datasets
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")

In [5]:
# Assign labels: 1 for Real, 0 for Fake
true_df['label'] = 1
fake_df['label'] = 0

In [6]:
# Merge datasets
df = pd.concat([true_df, fake_df], axis=0).sample(frac=1).reset_index(drop=True)

In [7]:
# Remove unnecessary columns
df.drop(columns=['title', 'subject', 'date'], inplace=True)

In [8]:
# Text Preprocessing Function
def clean_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    return text

In [9]:
# Apply text cleaning
df['text'] = df['text'].apply(clean_text)

In [10]:
# Split into training and testing sets
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Create a pipeline with TF-IDF Vectorizer and Naïve Bayes Classifier
model = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000)),  # Convert text to numerical format
    ('nb', MultinomialNB())  # Apply Naïve Bayes
])

In [12]:
# Train the model
model.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer(max_features=5000)),
                ('nb', MultinomialNB())])

In [13]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.9425389755011135

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94      4681
           1       0.94      0.94      0.94      4299

    accuracy                           0.94      8980
   macro avg       0.94      0.94      0.94      8980
weighted avg       0.94      0.94      0.94      8980



In [14]:
# Save the trained model
joblib.dump(model, "../Model/model.pkl")  # Save inside Model folder
print("✅ Model saved successfully as model.pkl!")

✅ Model saved successfully as model.pkl!
